# Chapter 1: GemPy Basic

In this first example, we will show how to construct a first basic model and the main objects and functions. First we import gempy:

In [1]:
# These two lines are necessary only if gempy is not installed
import sys, os
sys.path.append("../..")

# Importing gempy
import gempy as gp

# Embedding matplotlib figures into the notebooks
%matplotlib inline

# Aux imports
import numpy as np

WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


All data get stored in a python object InputData.  This object can be easily stored in a Python pickle. However, these files have the limitation that all dependecies must have the same versions as those when the pickle were created. For these reason to have more stable tutorials we will generate the InputData from raw data---i.e. csv files exported from Geomodeller.

These csv files can be found in the input_data folder in the root folder of GemPy. These tables contains uniquely the XYZ (and poles, azimuth and polarity in the foliation case) as well as their respective formation name (but not necessary the formation order).


In [2]:
# Importing the data from csv files and settign extent and resolution
geo_data = gp.create_data([0,2000,0,2000,-2000,0],[ 10,10,10],
                         path_f = os.pardir+"/input_data/FabLessPoints_Foliations.csv",
                         path_i = os.pardir+"/input_data/FabLessPoints_Points.csv")

here
2
3
there


../../gempy/data_management.py:66: UserWarning: path_f is deprecated use instead path_o
  warnings.warn('path_f is deprecated use instead path_o')


In [3]:
# Assigning series to formations as well as their order (timewise)
gp.set_series(geo_data, {"fault":'MainFault', 
                      "Rest": ('SecondaryReservoir','Seal', 'Reservoir', 'Overlying'), 
                               },
                       order_series = ["fault", 'Rest'],
                       order_formations=['MainFault', 
                                         'SecondaryReservoir', 'Seal','Reservoir', 'Overlying',
                                         ]) 
#geo_data = gp.select_series(geo_data, ['Rest'])


1
there


In [4]:
%debug

ERROR:root:No traceback has been produced, nothing to debug.


In [5]:
import theano.tensor as T
import theano
interp_data = gp.InterpolatorData(geo_data, u_grade=[1],
                                  output='geology', dtype='float64', compile_theano=False)
print(interp_data)
the = interp_data.interpolator.tg

In [6]:
input_data_T = the.input_parameters_list()
input_data_T

[Position of the dips,
 Angle of every dip,
 Azimuth,
 Polarity,
 Reference points for every layer,
 Rest of the points of the layers]

In [7]:
geo_data.interfaces.head()

,X,Y,Z,formation,series,formation_number,order_series,isFault,annotations,formation_value
0,1000.0,1000.0,-1000.0,MainFault,fault,1,1,True,"${\bf{x}}_{\alpha \,{\bf{1}},0}$",1
1,900.0,1000.0,-1300.0,MainFault,fault,1,1,True,"${\bf{x}}_{\alpha \,{\bf{1}},1}$",1
2,1100.0,1000.0,-700.0,MainFault,fault,1,1,True,"${\bf{x}}_{\alpha \,{\bf{1}},2}$",1
3,1200.0,1000.0,-400.0,MainFault,fault,1,1,True,"${\bf{x}}_{\alpha \,{\bf{1}},3}$",1
4,800.0,1000.0,-1600.0,MainFault,fault,1,1,True,"${\bf{x}}_{\alpha \,{\bf{1}},4}$",1


In [13]:
geo_data.formations

,value,formation_number
MainFault,1,1
SecondaryReservoir,5,2
Seal,4,3
Reservoir,3,4
Overlying,2,5
basement,6,6


In [8]:
th_fn = theano.function(input_data_T,
                         the.compute_grad3(0),
#                         mode=theano.compile.MonitorMode(
#                         pre_func=inspect_inputs,
#                         post_func=inspect_outputs),
                                    on_unused_input='ignore',
                                    allow_input_downcast=False,
                                    profile=False)

In [9]:
i = interp_data.get_input_data()

../../gempy/interpolator.py:540: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 2 but corresponding boolean dimension is 1
  n_universal_eq[u_grade == 0] = 0
../../gempy/interpolator.py:541: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 2 but corresponding boolean dimension is 1
  n_universal_eq[u_grade == 1] = 3
../../gempy/interpolator.py:542: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 2 but corresponding boolean dimension is 1
  n_universal_eq[u_grade == 2] = 9


In [10]:
th_fn(*i)


array([[  1.92552315e-02,  -2.36644032e-14,  -6.41840941e-03],
       [  3.03174740e-05,  -5.24590301e-16,  -1.01058092e-05],
       [ -4.62130055e-03,   1.37863103e-16,   1.54043347e-03],
       [ -9.41962683e-03,   8.15806916e-17,   3.13987587e-03],
       [ -8.37078725e+01,  -4.01847982e-02,   1.48841720e+03],
       [ -1.20723989e+02,   4.20941082e-01,  -6.80151404e+01],
       [  6.42377209e+02,  -5.89352691e-01,   1.10233970e+03],
       [ -1.42001964e+02,  -6.10399369e-02,  -1.37066931e+03],
       [ -1.57088757e+01,   8.07863692e-01,  -1.42619743e+02],
       [  3.24091780e+02,  -1.99503598e+00,   1.02381677e+03],
       [  1.38868247e+01,   1.77832071e+00,  -2.44967788e+02],
       [ -3.97948925e+01,  -2.33363247e-01,   1.03641122e+02],
       [  2.07434496e+02,   2.82546013e-01,  -2.99163353e+02],
       [ -9.14938175e+02,   6.08468828e-01,  -1.80492916e+03],
       [ -1.66740335e+01,  -1.55102429e+02,   1.81010885e+02],
       [  3.30048427e+02,   1.32589787e+02,  -2.3481030

In [11]:
tyito

NameError: name 'tyito' is not defined

In [ ]:
# import pandas as pn
# gp.set_orientations(geo_data,pn.DataFrame(geo_data.orientations.iloc[0,:]).T, append=True)
# geo_data.orientations.set_value(2, 'formation', 'Overlying')

In [ ]:
# # Assigning series to formations as well as their order (timewise)
# gp.set_series(geo_data, {"fault":'MainFault', 
#                       "Rest": ('SecondaryReservoir','Seal', 'Reservoir'), 
#                                "Rist": ('Overlying')},
#                        order_series = ["fault", 'Rest', 'Rist'],
#                        order_formations=['MainFault', 
#                                          'SecondaryReservoir', 'Seal','Reservoir', 'Overlying',
#                                          ]) 

# geo_data =gp.select_series(geo_data,['Rest', 'Rist'])

In [ ]:
geo_data.orientations

In [ ]:
gp.get_sequential_pile(geo_data)

## The ins and outs of Input data objects

As we have seen objects DataManagement.InputData (usually called geo_data in the tutorials) aim to have all the original geological properties, measurements and geological relations stored. 

Once we have the data ready to generate a model, we will need to create the next object type towards the final geological model:

In [ ]:
geo_data.interfaces.drop(39, inplace=True)

In [ ]:
import theano.tensor as T
import theano
interp_data = gp.InterpolatorData(geo_data, u_grade=[1, 1],
                                  output='geology', dtype='float64',
                                   verbose=['scalar_field_iter', 'block_series', 'yet_simulated'],
                                  compile_theano=True)


In [ ]:
interp_data.interpolator.tg.len_series_i.get_value()

In [ ]:
interp_data.interpolator.tg.n_formations_per_serie.get_value()

In [ ]:
interp_data.interpolator.tg.n_formations_per_serie.set_value(np.array([0, 3, 4], dtype='int32'))

In [ ]:
interp_data.interpolator.tg.npf.get_value()

In [ ]:
geo_data.interfaces.shape

In [ ]:
interp_data.interpolator.tg.npf.get_value()

In [ ]:
import matplotlib.pyplot as plt
x = np.linspace(-10,50)
sigm = 1. / (1 + np.exp(-1 * (x - 0))) + 1. / (1 + np.exp(1 * (x - 30))) -1
plt.plot(x, sigm)

In [ ]:
interp_data.update_interpolator(geo_data)

In [ ]:
interp_data.interpolator.tg.n_formation_float.get_value()

In [ ]:
interp_data.interpolator.tg.n_formation_float.set_value(np.array([ 1.,  2.,  3.,  4.,  5., 6.], dtype='float32'))

In [ ]:
sol = gp.compute_model(interp_data)

In [ ]:
%matplotlib notebook
gp.plot_section(geo_data,sol[0][0].astype(float), 30, plot_data = True, direction='y')

In [ ]:
interp_data.interpolator.tg.n_formation_float.set_value(np.array([ 1.,  2.,  3.,  4.,  5., 6.], dtype='float32'))

In [ ]:
interp_data.interpolator.tg.n_formations_per_serie.get_value()

In [ ]:
interp_data.interpolator.tg.len_series_i.set_value(np.array([ 0,  4, 35], dtype='int32'))

In [ ]:
interp_data.interpolator.tg.len_series_i.get_value()

In [ ]:
interp_data.interpolator.tg.npf.get_value()

In [ ]:
asa = interp_data.get_input_data()

In [ ]:
interp_data.th_fn(*asa)

In [ ]:
interp_data.interpolator.pandas_ref_layer_points

In [ ]:

interp_data.geo_data_res.interfaces.drop(39, inplace=True)